# Objetivo: Comprar as ações com as melhores perfomaces acumuladas ao longo dos últimos 11 meses e vender as com as piores

# Importando Pacotes

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime as dt
import csv 
from pandas.tseries.offsets import MonthEnd 

# Obtendo e Limpando os dados

In [ ]:
tickers = pd.read_csv('tickers.csv', sep=";")
tickers = pd.DataFrame(tickers)
tickers = tickers['0'].to_list()


In [ ]:
df2 = yf.download(tickers, start='2009-12-31',end='2020-12-31') # baixo todas as ações dela num período de 10 anos

[*********************100%***********************]  75 of 75 completed


In [ ]:
prices = df2['Adj Close'] # separo somente a coluna de fechamento de preços
prices

,ABEV3.SA,AZUL4.SA,B3SA3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BEEF3.SA,BPAC11.SA,BRAP4.SA,BRDT3.SA,BRFS3.SA,BRKM5.SA,BRML3.SA,BTOW3.SA,CCRO3.SA,CIEL3.SA,CMIG4.SA,COGN3.SA,CPFE3.SA,CRFB3.SA,CSAN3.SA,CSNA3.SA,CVCB3.SA,CYRE3.SA,ECOR3.SA,EGIE3.SA,ELET3.SA,ELET6.SA,EMBR3.SA,ENBR3.SA,ENGI11.SA,EQTL3.SA,FLRY3.SA,GGBR4.SA,GNDI3.SA,GOAU4.SA,GOLL4.SA,HAPV3.SA,HGTX3.SA,HYPE3.SA,IGTA3.SA,IRBR3.SA,ITSA4.SA,ITUB4.SA,JBSS3.SA,KLBN11.SA,LAME4.SA,LREN3.SA,MGLU3.SA,MRFG3.SA,MRVE3.SA,MULT3.SA,NTCO3.SA,PCAR3.SA,PETR3.SA,PETR4.SA,QUAL3.SA,RADL3.SA,RAIL3.SA,RENT3.SA,SANB11.SA,SBSP3.SA,SULA11.SA,SUZB3.SA,TAEE11.SA,TIMP3.SA,TOTS3.SA,UGPA3.SA,USIM5.SA,VALE3.SA,VIVT4.SA,VVAR3.SA,WEGE3.SA,YDUQ3.SA
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-04,2.582460,NaN,NaN,16.223814,8.056501,10.754330,NaN,5.259252,NaN,24.015770,NaN,21.222513,8.802499,5.415267,40.834820,4.775975,2.710821,7.773108,NaN,9.709170,NaN,16.387367,16.049044,NaN,16.557974,NaN,8.112115,25.281479,NaN,8.911546,5.162889,NaN,0.833473,NaN,25.081799,NaN,30.342358,25.250458,NaN,NaN,17.021349,13.888401,NaN,3.940176,12.261915,8.649340,NaN,7.363498,NaN,NaN,19.335154,7.326664,7.479765,NaN,29.847929,36.164825,31.196165,NaN,1.179861,NaN,3.966986,14.851018,6.588534,8.175477,14.911905,NaN,5.561658,18.011520,8.244670,23.776321,33.246456,17.286230,3.313528,3.751335,NaN
2010-01-05,2.597651,NaN,NaN,16.061029,7.950834,10.706059,NaN,5.467084,NaN,24.499760,NaN,21.444983,8.722194,5.515550,39.860256,4.841481,2.691948,7.892072,NaN,9.592169,NaN,16.393803,16.197515,NaN,16.400602,NaN,8.176021,25.074085,NaN,8.703883,5.202695,NaN,0.840046,NaN,25.505482,NaN,30.707117,25.039238,NaN,NaN,17.245752,13.745355,NaN,3.927432,12.341385,8.568838,NaN,7.106353,NaN,NaN,20.730476,7.230773,7.446469,NaN,29.847929,35.947750,30.928682,NaN,1.151195,NaN,4.019883,14.994837,6.598199,8.057549,14.911905,NaN,5.964453,17.710026,8.244670,23.846254,33.556385,17.166214,3.230424,3.743146,NaN
2010-01-06,2.624151,NaN,NaN,16.082731,7.903974,10.620417,NaN,5.738181,NaN,24.842840,NaN,21.736973,8.932220,5.766258,40.299274,5.002269,2.685087,7.743392,NaN,9.655989,NaN,15.924658,16.217503,NaN,16.421125,NaN,8.093324,24.752951,NaN,8.667766,5.279191,NaN,0.831125,NaN,25.674946,NaN,31.012478,25.327267,NaN,NaN,17.918959,13.678006,NaN,3.879655,12.234464,8.703005,NaN,6.970778,NaN,NaN,21.278639,7.353333,7.546353,NaN,29.847929,36.303753,31.346632,NaN,1.162830,NaN,4.333177,14.919800,6.596277,8.043385,14.911905,NaN,5.886992,17.815582,8.244670,23.822941,34.266640,17.322266,3.198460,3.763613,NaN
2010-01-07,2.636473,NaN,NaN,16.088154,7.885812,10.591830,NaN,5.864691,NaN,25.118530,NaN,21.602562,8.709840,5.691042,40.755787,4.955819,2.748569,7.669055,NaN,9.536288,NaN,15.076400,16.046194,NaN,16.106390,NaN,8.145951,25.328272,NaN,8.550391,5.317460,NaN,0.821734,NaN,25.098745,NaN,30.401733,25.596092,NaN,NaN,18.135054,13.572807,NaN,3.857359,12.109151,8.899784,NaN,6.966101,NaN,NaN,22.325129,7.086936,7.513058,NaN,29.847929,36.086678,31.054062,NaN,1.154934,NaN,4.150087,14.813501,6.491481,7.925458,14.911905,NaN,5.755310,17.830605,8.244670,24.046719,34.408699,17.234257,3.206984,3.827053,NaN
2010-01-08,2.631408,NaN,NaN,16.180399,7.896163,10.580409,NaN,6.235187,NaN,25.486116,NaN,21.690622,8.586296,5.640900,39.781223,4.954628,2.714254,7.661599,NaN,9.655989,NaN,15.211329,16.408804,NaN,16.079021,NaN,8.078286,25.756451,NaN,8.532333,5.386321,NaN,0.833943,NaN,25.030958,NaN,30.452633,26.834614,NaN,NaN,18.617098,13.467523,NaN,3.828690,11.925772,8.908729,NaN,6.989473,NaN,NaN,22.773624,6.879096,7.180113,NaN,29.847929,35.817501,30.886881,NaN,1.175293,NaN,4.188739,14.663425,6.600177,7.939557,14.911905,NaN,5.693342,18.003967,8.244670,23.706390,34.744453,17.066153,3.204853,3.966224,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
prices.index = pd.to_datetime(prices.index) # a coluna de datas vem como uma string por default, é necessario transforma-la para index

In [ ]:
mtl_ret = prices.pct_change().resample('M').agg(lambda x: (x+1).prod() -1) # transformando os dados diários em variações percentuais mensais

In [ ]:
mtl_ret

,ABEV3.SA,AZUL4.SA,B3SA3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BEEF3.SA,BPAC11.SA,BRAP4.SA,BRDT3.SA,BRFS3.SA,BRKM5.SA,BRML3.SA,BTOW3.SA,CCRO3.SA,CIEL3.SA,CMIG4.SA,COGN3.SA,CPFE3.SA,CRFB3.SA,CSAN3.SA,CSNA3.SA,CVCB3.SA,CYRE3.SA,ECOR3.SA,EGIE3.SA,ELET3.SA,ELET6.SA,EMBR3.SA,ENBR3.SA,ENGI11.SA,EQTL3.SA,FLRY3.SA,GGBR4.SA,GNDI3.SA,GOAU4.SA,GOLL4.SA,HAPV3.SA,HGTX3.SA,HYPE3.SA,IGTA3.SA,IRBR3.SA,ITSA4.SA,ITUB4.SA,JBSS3.SA,KLBN11.SA,LAME4.SA,LREN3.SA,MGLU3.SA,MRFG3.SA,MRVE3.SA,MULT3.SA,NTCO3.SA,PCAR3.SA,PETR3.SA,PETR4.SA,QUAL3.SA,RADL3.SA,RAIL3.SA,RENT3.SA,SANB11.SA,SBSP3.SA,SULA11.SA,SUZB3.SA,TAEE11.SA,TIMP3.SA,TOTS3.SA,UGPA3.SA,USIM5.SA,VALE3.SA,VIVT4.SA,VVAR3.SA,WEGE3.SA,YDUQ3.SA
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-31,-0.019609,0.000000,0.000000,-0.060201,-0.087070,-0.168285,0.000000,0.183848,0.000000,-0.009694,0.000000,-0.013977,-0.069474,-0.050925,-0.182972,0.002993,-0.050633,-0.000640,0.000000,-0.006028,0.000000,-0.164705,-0.021526,0.000000,-0.105785,0.000000,-0.062558,5.953975e-02,0.000000,0.012159,0.047722,0.000000,-0.042255,0.000000,-0.143919,0.000000,-0.123847,-0.121673,0.000000,0.000000,0.060059,-0.136664,0.000000,-0.092158,-0.099451,-0.034126,0.000000,-0.160000,0.000000,0.000000,0.159794,-0.116359,-0.109793,0.000000,0.000000,-0.082353,-0.084405,0.000000,-0.049294,0.000000,0.012309,-0.051789,-0.076586,-0.057515,0.000000,0.000000,-0.025069,-0.030128,0.000000,-0.027451,-0.050107,-0.037500,-0.144694,-0.015275,0.000000
2010-02-28,-0.005800,0.000000,0.000000,0.076227,-0.004265,-0.009578,0.000000,-0.042089,0.000000,0.015714,0.000000,-0.023256,-0.036199,0.146341,-0.013159,-0.051467,-0.056666,-0.052313,0.000000,0.023705,0.000000,0.105165,0.075454,0.000000,0.042052,0.000000,0.023727,-4.155854e-01,0.013781,-0.018018,-0.002832,0.000000,0.033531,0.000000,0.043805,0.000000,0.048181,0.062771,0.000000,0.000000,-0.006448,0.003859,0.000000,0.029384,0.010242,-0.027837,0.000000,-0.017384,0.000000,0.000000,-0.064000,0.078185,0.041329,0.000000,0.000000,0.012559,0.012877,0.000000,0.029627,0.000000,-0.002027,-0.050178,-0.031895,-0.030610,0.000000,0.000000,0.050000,-0.055217,0.000000,0.034678,0.031486,-0.052427,-0.172932,-0.021608,0.000000
2010-03-31,-0.010329,0.000000,0.000000,0.002687,0.025886,0.000994,0.000000,0.151514,0.000000,0.114633,0.000000,0.086168,0.024257,-0.106383,0.027140,0.035124,0.272361,-0.006392,0.000000,0.031275,0.000000,-0.079016,0.206086,0.000000,-0.066076,0.000000,-0.034766,1.196570e-01,0.110492,0.072375,-0.029787,0.000000,-0.109847,0.000000,0.092627,0.000000,0.110502,-0.062212,0.000000,0.000000,0.008345,0.047201,0.000000,0.055957,0.068495,-0.124449,0.000000,0.024000,0.000000,0.000000,-0.034188,-0.052673,-0.056975,0.000000,1.306676,0.025840,0.022537,0.000000,0.028419,0.000000,-0.045685,0.027583,0.051726,0.015788,0.000000,0.000000,-0.047619,0.102298,-0.483974,0.187062,0.132805,-0.022589,0.217273,0.075877,0.000000
2010-04-30,0.065855,0.000000,0.000000,0.004690,-0.011836,0.000995,0.000000,-0.018421,0.000000,-0.088282,0.000000,-0.052192,-0.039725,0.054285,-0.000781,0.043578,0.007762,0.026589,0.000000,0.006785,0.000000,0.019374,-0.095038,0.000000,-0.002849,0.010527,0.108054,-5.724820e-02,-0.052103,-0.023254,0.169650,0.000000,0.142215,0.000000,-0.015571,0.000000,-0.066886,0.035874,0.000000,0.000000,0.103448,-0.018030,0.000000,-0.015600,-0.033336,0.027673,0.000000,-0.024152,0.000000,0.000000,-0.103245,-0.008864,0.014936,0.000000,0.000000,-0.066004,-0.069524,0.000000,0.085385,0.000000,0.037233,-0.085987,0.045742,0.044153,0.000000,0.000000,-0.109368,0.067062,0.000000,-0.078956,-0.064274,-0.053253,-0.051531,-0.065789,0.000000
2010-05-31,0.001223,0.000000,0.000000,-0.120863,-0.047206,0.000994,0.000000,-0.121984,0.000000,-0.065667,0.000000,0.026432,-0.141607,0.119747,-0.202503,-0.076789,-0.087678,-0.056621,0.000000,0.032306,0.000000,-0.061086,-0.107233,0.000000,-0.037977,-0.011458,-0.061339,-6.987741e-02,-0.080514,-0.044117,0.001505,0.000000,-0.019233,0.000000,-0.117044,0.000000,-0.067861,

# Calculando o ProfitMomentum para apenas 1 mês

In [ ]:
past_11 = (mtl_ret+1).rolling(11).apply(np.prod)-1 # calcula a média com base nos últimos 11 meses, por isso perde 11 dados no começo

In [ ]:
past_11.head(14) # verificando

,ABEV3.SA,AZUL4.SA,B3SA3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BEEF3.SA,BPAC11.SA,BRAP4.SA,BRDT3.SA,BRFS3.SA,BRKM5.SA,BRML3.SA,BTOW3.SA,CCRO3.SA,CIEL3.SA,CMIG4.SA,COGN3.SA,CPFE3.SA,CRFB3.SA,CSAN3.SA,CSNA3.SA,CVCB3.SA,CYRE3.SA,ECOR3.SA,EGIE3.SA,ELET3.SA,ELET6.SA,EMBR3.SA,ENBR3.SA,ENGI11.SA,EQTL3.SA,FLRY3.SA,GGBR4.SA,GNDI3.SA,GOAU4.SA,GOLL4.SA,HAPV3.SA,HGTX3.SA,HYPE3.SA,IGTA3.SA,IRBR3.SA,ITSA4.SA,ITUB4.SA,JBSS3.SA,KLBN11.SA,LAME4.SA,LREN3.SA,MGLU3.SA,MRFG3.SA,MRVE3.SA,MULT3.SA,NTCO3.SA,PCAR3.SA,PETR3.SA,PETR4.SA,QUAL3.SA,RADL3.SA,RAIL3.SA,RENT3.SA,SANB11.SA,SBSP3.SA,SULA11.SA,SUZB3.SA,TAEE11.SA,TIMP3.SA,TOTS3.SA,UGPA3.SA,USIM5.SA,VALE3.SA,VIVT4.SA,VVAR3.SA,WEGE3.SA,YDUQ3.SA
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
formation = dt.datetime(2010,12,31) #  Para nao ter mais dados NaN, é criada uma nova base de dados para que começa a partir do 3º dado valido

In [ ]:
formation # ela é em formato datetime

datetime.datetime(2010, 12, 31, 0, 0)

In [ ]:
end_measurement = formation - MonthEnd(1) # Formation, sera utilizado na previsão que começa a partir do mes anterior

In [ ]:
end_measurement

Timestamp('2010-11-30 00:00:00')

In [ ]:
ret_12 = past_11.loc[end_measurement]

In [ ]:
ret_12 # todas as ações desta data

ABEV3.SA    0.564864
AZUL4.SA    0.000000
B3SA3.SA    0.000000
BBAS3.SA    0.154115
BBDC3.SA    0.060099
              ...   
VALE3.SA    0.058515
VIVT4.SA    0.005694
VVAR3.SA    0.221865
WEGE3.SA    0.269582
YDUQ3.SA    0.000000
Name: 2010-11-30 00:00:00, Length: 75, dtype: float64

In [ ]:
ret_12 = ret_12.reset_index() # mudando para o formato dataframe

In [ ]:
ret_12 

,index,2010-11-30 00:00:00
0,ABEV3.SA,0.564864
1,AZUL4.SA,0.000000
2,B3SA3.SA,0.000000
3,BBAS3.SA,0.154115
4,BBDC3.SA,0.060099
...,...,...
70,VALE3.SA,0.058515
71,VIVT4.SA,0.005694
72,VVAR3.SA,0.221865
73,WEGE3.SA,0.269582


In [ ]:
ret_12['decile'] = pd.qcut(ret_12.iloc[:,1],10,labels=False,duplicates='drop') # adciona uma coluna dos decis (10º percentil em 10º percentil)

In [ ]:
ret_12 # as ações vencedoras no período são as do decil de número nove

,index,2010-11-30 00:00:00,decile
0,ABEV3.SA,0.564864,7
1,AZUL4.SA,0.000000,2
2,B3SA3.SA,0.000000,2
3,BBAS3.SA,0.154115,5
4,BBDC3.SA,0.060099,4
...,...,...,...
70,VALE3.SA,0.058515,4
71,VIVT4.SA,0.005694,3
72,VVAR3.SA,0.221865,5
73,WEGE3.SA,0.269582,5


In [ ]:
winners =  ret_12[ret_12.decile == 7]
losers = ret_12[ret_12.decile == 0]

In [ ]:
winners.head() # são os 10% com mais retorno no mês

,index,2010-11-30 00:00:00,decile
0,ABEV3.SA,0.564864,7
13,BRML3.SA,0.672131,7
15,CCRO3.SA,0.419805,7
25,ECOR3.SA,0.463158,7
54,PCAR3.SA,1.306676,7


In [ ]:
winnerret =  mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(winners['index'])] # isin(winners) significa que são apenas os do 9º decil
loserret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(losers['index'])]

In [ ]:
Momentumprofit = winnerret.mean() - loserret.mean() # Lucro do momentum

In [ ]:
Momentumprofit # lucro caso eu tivesse comprado as ações que mais renderam em novembro de 2010 e vendido elas em dezembro deste ano

-0.012756224808797234

# Fazendo uma função que itera para todos os meses

In [ ]:
def momentum(formation):
  end_measurement = formation - MonthEnd(1)
  ret_12 = past_11.loc[end_measurement]
  ret_12 = ret_12.reset_index()
  ret_12['decile'] = pd.qcut(ret_12.iloc[:,1],10,labels=False,duplicates='drop')
  winners =  ret_12[ret_12.decile == 7]
  losers = ret_12[ret_12.decile == 0]
  winnerret =  mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(winners['index'])] # isin(winners) significa que são apenas os do 9º decil
  loserret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(losers['index'])]
  Momentumprofit = winnerret.mean() - loserret.mean()
  return Momentumprofit

In [ ]:
formation

datetime.datetime(2010, 12, 31, 0, 0)

In [ ]:
momentum(formation) # testando para o mês que já calculei na mão

-0.012756224808797234

In [ ]:
for i in range((12*10)): # loop para 10 anos compostos por 12 meses
    print(formation + MonthEnd(i)) # printa todos os meses

2010-12-31 00:00:00
2011-01-31 00:00:00
2011-02-28 00:00:00
2011-03-31 00:00:00
2011-04-30 00:00:00
2011-05-31 00:00:00
2011-06-30 00:00:00
2011-07-31 00:00:00
2011-08-31 00:00:00
2011-09-30 00:00:00
2011-10-31 00:00:00
2011-11-30 00:00:00
2011-12-31 00:00:00
2012-01-31 00:00:00
2012-02-29 00:00:00
2012-03-31 00:00:00
2012-04-30 00:00:00
2012-05-31 00:00:00
2012-06-30 00:00:00
2012-07-31 00:00:00
2012-08-31 00:00:00
2012-09-30 00:00:00
2012-10-31 00:00:00
2012-11-30 00:00:00
2012-12-31 00:00:00
2013-01-31 00:00:00
2013-02-28 00:00:00
2013-03-31 00:00:00
2013-04-30 00:00:00
2013-05-31 00:00:00
2013-06-30 00:00:00
2013-07-31 00:00:00
2013-08-31 00:00:00
2013-09-30 00:00:00
2013-10-31 00:00:00
2013-11-30 00:00:00
2013-12-31 00:00:00
2014-01-31 00:00:00
2014-02-28 00:00:00
2014-03-31 00:00:00
2014-04-30 00:00:00
2014-05-31 00:00:00
2014-06-30 00:00:00
2014-07-31 00:00:00
2014-08-31 00:00:00
2014-09-30 00:00:00
2014-10-31 00:00:00
2014-11-30 00:00:00
2014-12-31 00:00:00
2015-01-31 00:00:00


In [ ]:
profits = []
dates = []

for i in range((12*10)-1):
  profits.append(momentum(formation + MonthEnd(i)))
  dates.append(formation + MonthEnd(i))

In [ ]:
len(profits) # agora todos os lucros de todos os meses estão printados nesta lista

119

In [ ]:
len(dates)

119

# Benchmark (verificando como esta estratégia performa em relação ao mercado)

In [ ]:
BVSP = yf.download('^BVSP', start='2011-2-28',end='2020-12-31')

[*********************100%***********************]  1 of 1 completed


In [ ]:
BVSP

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2011-02-28,66909.0,67458.0,66687.0,67383.0,67383.0,2640400
2011-03-01,67381.0,67708.0,66243.0,66243.0,66243.0,2166400
2011-03-02,66242.0,67282.0,66101.0,67282.0,67282.0,2518600
2011-03-03,67284.0,68188.0,67284.0,68146.0,68146.0,2254600
2011-03-04,68138.0,68220.0,67514.0,68012.0,68012.0,1840000
...,...,...,...,...,...,...
2020-12-22,115825.0,116903.0,115648.0,116348.0,116348.0,6947900
2020-12-23,116636.0,118311.0,116636.0,117857.0,117857.0,6483300
2020-12-28,117806.0,119213.0,117805.0,119051.0,119051.0,7171700


In [ ]:
BVSP = BVSP['Adj Close'] # pega os dados do ibovespa americano
len(SP)

2430

In [ ]:
BVSP_mtl = BVSP.pct_change().resample('M').agg(lambda x: (x+1).prod() -1)

In [ ]:
len(BVSP_mtl)

119

In [ ]:
frame = pd.DataFrame(profits) # converto para dataframe

In [ ]:
frame # o profits, calculado no loop

,0
0,-0.012756
1,-0.028663
2,-0.018891
3,0.035243
4,-0.067860
...,...
114,0.048644
115,-0.015774
116,0.049707
117,0.103780


In [ ]:
frame['Ibovespa'] = BVSP_mtl.values # adciono uma coluna com as variações percentuais do SP500 neste dataframe

In [ ]:
frame

,0,Ibovespa
0,-0.012756,0.000000
1,-0.028663,0.017868
2,-0.018891,-0.035779
3,0.035243,-0.022878
4,-0.067860,-0.034293
...,...,...
114,0.048644,-0.034427
115,-0.015774,-0.047963
116,0.049707,-0.006881
117,0.103780,0.158975


In [ ]:
frame['excess'] = frame.iloc[:,0] - frame.iloc[:,1] # a performace é a subtração destas duas colunas

In [ ]:
frame['outperformed'] = ['Yes' if i > 0 else 'No' for i in frame.excess]

In [ ]:
frame # outperformed = performace positiva

,0,Ibovespa,excess,outperformed
0,-0.012756,0.000000,-0.012756,No
1,-0.028663,0.017868,-0.046531,No
2,-0.018891,-0.035779,0.016888,Yes
3,0.035243,-0.022878,0.058121,Yes
4,-0.067860,-0.034293,-0.033567,No
...,...,...,...,...
114,0.048644,-0.034427,0.083071,Yes
115,-0.015774,-0.047963,0.032188,Yes
116,0.049707,-0.006881,0.056588,Yes
117,0.103780,0.158975,-0.055195,No


In [ ]:
frame[frame.outperformed == "Yes"].shape # contando elas

(54, 4)

In [ ]:
frame.shape

(119, 4)

In [ ]:
54/119 # taxa de acerto desta estratégia

0.453781512605042